In [1]:
import sys
sys.path.append('../edaf')

In [2]:
from plot_helpers import *
from decomp import *
from loguru import logger
import os, sys, gzip, json
import sqlite3
import pandas as pd
from edaf.core.uplink.preprocess import preprocess_ul
from edaf.core.uplink.analyze_packet import ULPacketAnalyzer
from edaf.core.uplink.analyze_channel import ULChannelAnalyzer
from edaf.core.uplink.analyze_scheduling import ULSchedulingAnalyzer
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from pathlib import Path
import ijson

%load_ext autoreload
%autoreload 2

In [3]:
# Remove default handler
logger.remove()
# Add a new handler with level WARNING
logger.add(sys.stdout, level="ERROR")

16

In [4]:
MEAS_DIR = '../meas-oai5g/s18/'
DB_FILE = MEAS_DIR+'res_db.sql'
PLOTS_DIR = './plots/'

In [5]:
analyzer = ULPacketAnalyzer(DB_FILE)
uids_arr = range(analyzer.first_ueipid, analyzer.last_ueipid+1)
packets = analyzer.figure_packettx_from_ueipids(uids_arr)

In [18]:
send_ts = analyzer.nlmt_df['timestamps.client.send.wall'].to_numpy()
send_ts = (send_ts)/1e6
receive_ts = analyzer.nlmt_df['timestamps.server.receive.wall'].to_numpy()
receive_ts = (receive_ts)/1e6
idt = (send_ts[1:-1]-send_ts[0:-2])
e2e_delays = receive_ts-send_ts
ran_delays = {packet['id']: get_ran_delay(packet) for packet in packets if get_ran_delay(packet)!=None}.values()
queueing_delays = {packet['id']: get_queueing_delay(packet) for packet in packets if get_queueing_delay(packet)!=None}.values()
retx_delays = {packet['id']: get_retx_delay(packet) for packet in packets if get_retx_delay(packet)!=None}.values()

In [19]:
fig, ax  = plot_ccdf(ran_delays[100:-100], 's18', figsize=(10, 6))
plt.savefig(PLOTS_DIR+"ran_delay_ccdf_plot.png", dpi=300, bbox_inches='tight')  # You can change the file format (e.g., .pdf)
plt.show()

TypeError: 'dict_values' object is not subscriptable

In [ ]:
# Create a figure and axis
fig, ax = plt.subplots(figsize=(10, 6))
labels = ['mcsi20', 'mcsi25']

# Sort the delays in ascending order
for ind, DB_FILE in enumerate(DB_FILES):
    sorted_delays = np.sort(list(retx_delays[ind].values())[100:-100])
    ccdf = 1.0 - np.arange(1, len(sorted_delays) + 1) / len(sorted_delays)

    # Plot CCDF
    ax.plot(sorted_delays, ccdf, linestyle='-', linewidth=4, label=labels[ind])
    
    # Set the y-axis to a logarithmic scale
    ax.set_yscale('log')
    
    # Label the axes
    ax.set_xlabel('Delay (ms)')
    ax.set_ylabel('Probability')
    #ax.set_xlim(15,30)
    # Add title
    
    # Add grid
    ax.grid(True)
ax.legend()
plt.savefig("ran_ccdf_plot.png", dpi=300, bbox_inches='tight')  # You can change the file format (e.g., .pdf)